In [2]:
# Importação das bibliotecas
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

# Hipoteses Levantadas


In [7]:
#Importação dos dados
ocorrencia = pd.read_csv('/content/ocorrencia.csv',  sep = ';',error_bad_lines=False)
recomendacao = pd.read_csv('/content/recomendacao.csv',  sep = ';',error_bad_lines=False)
ocorrencia_tipo = pd.read_csv('/content/ocorrencia_tipo.csv',  sep = ';',error_bad_lines=False)
aeronave = pd.read_csv('/content/aeronave.csv',  sep = ';',error_bad_lines=False)
fator_contribuinte = pd.read_csv('/content/fator_contribuinte.csv',  sep = ';',error_bad_lines=False)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


##1) Qual é a porcentagem de classificação das ocorrências relatadas?

In [8]:
fig = px.pie(ocorrencia, values=ocorrencia["ocorrencia_classificacao"].value_counts(),\
             names= ocorrencia["ocorrencia_classificacao"].value_counts().index )
fig.show()

A porcentagem de ocorrências de acidentes foi 32,3%(equivalente a 1667 ocorrências).  O acidente representa justamente uma ocorrência mais significativa em termos de perdas materiais e humanas. Por esta razão, as análises apresentadas no restante deste trabalho consideram apenas os acidentes


##2) Qual são os tipos de danos mais relatados nas ocorrências de acidente?

In [16]:
#Selecionar amostras acidente
amostra_acidente = ocorrencia.loc[ocorrencia.ocorrencia_classificacao=='ACIDENTE']

In [17]:
#Ocorrência com aeronave
amostra_acidente = pd.merge(amostra_acidente , aeronave[['codigo_ocorrencia2','aeronave_nivel_dano','aeronave_tipo_veiculo','aeronave_motor_tipo']], how = 'inner', on = 'codigo_ocorrencia2')


In [18]:

#Nível de danos nas aeronaves com ocorrências do tipo incidente grave
fig = px.pie(amostra_acidente, values=amostra_acidente ["aeronave_nivel_dano"].value_counts(),\
             names= amostra_acidente ["aeronave_nivel_dano"].value_counts().index )
fig.write_html('Figura1.html')
fig.show()

17,9% das 1667 ocorrências de acidente tiveram aeronaves destruídas e 77,1 com danos substancial.  Cerca de 5% são ocorrência tiveram danos classificadas como leve, nenhum ou não identificada. 


##3) Qual são os tipos de aeronaves  e o tipo de motor mais relatadas nas ocorrências de acidente? 

In [ ]:
acidente_tipo_veículo = amostra_acidente['aeronave_tipo_veiculo'].value_counts()

fig = px.bar(acidente_tipo_veículo, x=acidente_tipo_veículo.index, 
             y=acidente_tipo_veículo, color=acidente_tipo_veículo.index, 
             orientation='v',            
             height=400,
             text_auto = True,    
             title='Número de ocorrências de acidente tipo de aeronave')

fig.update_xaxes(title= 'Ocorrências')
fig.update_yaxes(title= 'aeronaves')    
    
fig.update_layout(
    height=400,
    showlegend=True,    
    width = 800,  
)

O avião é um tipo de aeronave mais frequente em ocorrências aéreas no Brasil,1212 das 1667 ocorrências de acidentes foram relatadas em aviões. Seguido por Ultraleves 212 e Helicóptero 186 ocorrências. 



In [23]:
acidente_tipo_motor = amostra_acidente['aeronave_motor_tipo'].value_counts()

fig = px.bar(acidente_tipo_motor, x=acidente_tipo_motor.index, 
             y=acidente_tipo_motor, color=acidente_tipo_motor.index, 
             orientation='v',            
             height=400,
             text_auto = True,    
             title='Número de ocorrências de acidente tipo de motor')

fig.update_xaxes(title= 'Ocorrências')
fig.update_yaxes(title= 'aeronaves')    
    
fig.update_layout(
    height=400,
    showlegend=True,    
    width = 800,  
)

Os motores aeronáuticos são classificados em motores a pistão, a jato, turboélice e os turboeixo. É possível observar, no gráfico que os motores a pistão são encontrados em maior frequência nas ocorrências reportadas.

##4) Qual são os top 10 estados mais relatados nas ocorrências de acidente?



In [ ]:
# Top 10 estados com maiores ocorrências de acidentes
grupo_ocorrencias = amostra_acidente[['ocorrencia_uf','codigo_ocorrencia'	]].groupby('ocorrencia_uf').count() 
amostras_sele_estado =  grupo_ocorrencias.sort_values(by = 'codigo_ocorrencia', ascending = False) [0:10]

fig = px.bar(amostras_sele_estado, x=amostras_sele_estado.codigo_ocorrencia, 
             y=amostras_sele_estado.index, color=amostras_sele_estado.index, 
             orientation='h',
             text_auto = True,                
             height=400,
             title='Número de ocorrências por Estado')

fig.update_xaxes(title= 'Ocorrências')
fig.update_yaxes(title= ' Estados')    
    
fig.update_layout(
    height=400,
    showlegend=True,    
    width = 800,
   
)

A quantidade de registros de acidentes em São Paulo foi expressivamente maior que a quantidade de registros demais  estados. Esse número é devido à concentração de aeródromos no Estado de São Paulo.


##5) Quais são os tipos de ocorrências de acidente mais relatados?

In [ ]:
#Tipos de ocorrências
amostra_acidente_tipo= pd.merge(amostra_acidente, ocorrencia_tipo[['codigo_ocorrencia1','ocorrencia_tipo']], how = 'inner', on = 'codigo_ocorrencia1')

In [ ]:
#Ocorrência por cidade de São Paulo
group_tipo = amostra_acidente_tipo.groupby(['ocorrencia_uf', 'ocorrencia_tipo']).count() 
group_tipo  = group_tipo .reset_index()
fig = px.treemap(group_tipo,path=['ocorrencia_tipo','ocorrencia_uf'], values='codigo_ocorrencia')
fig.show()

No treemap é possível observar o tipo de ocorrência por estado. As causas dos acidentes mais relatadas foram falha do motor em voo (SP e MG),perda de controle em voo (SP e MT ), perda de controle no solo (SP e MG). 


##6) Qual é o número de ocorrências de acidente por ano?


In [ ]:
amostra_acidente['ocorrencia_dia'] = pd.to_datetime(amostra_acidente['ocorrencia_dia'],errors = 'coerce')
amostra_acidente['ano'] = amostra_acidente['ocorrencia_dia'].dt.year

In [ ]:
amostra_acidente.count()

codigo_ocorrencia                 1678
codigo_ocorrencia1                1678
codigo_ocorrencia2                1678
codigo_ocorrencia3                1678
codigo_ocorrencia4                1678
ocorrencia_classificacao          1678
ocorrencia_latitude               1676
ocorrencia_longitude              1676
ocorrencia_cidade                 1678
ocorrencia_uf                     1678
ocorrencia_pais                   1678
ocorrencia_aerodromo              1678
ocorrencia_dia                    1678
ocorrencia_hora                   1678
investigacao_aeronave_liberada    1592
investigacao_status               1493
divulgacao_relatorio_numero       1465
divulgacao_relatorio_publicado    1678
divulgacao_dia_publicacao         1035
total_recomendacoes               1678
total_aeronaves_envolvidas        1678
ocorrencia_saida_pista            1678
aeronave_nivel_dano               1678
aeronave_tipo_veiculo             1678
ano                               1678
dtype: int64

In [ ]:
grupo_ano = amostra_acidente[['ano', 'total_aeronaves_envolvidas','codigo_ocorrencia']].groupby(['ano']).count()

fig = px.bar(grupo_ano, x=grupo_ano.index, 
             y=grupo_ano.codigo_ocorrencia, 
             orientation='v', 
             text_auto = True,         
             height=400,
             title='Número de ocorrências por ano')

fig.update_xaxes(title= 'Ano')
fig.update_yaxes(title= 'Ocorrências')    
    
fig.update_layout(
    height=400,
    showlegend=True,    
    width = 800,
   
)

Em relação a  série temporal dos acidentes, ocorreu  redução dos casos registrados entre os anos de 2012 (206 acidentes) à 2021(142 acidentes). Em 2018 ocorreu um ligeiro aumento. 
